### Pseudobulk differential expression analysis (DESeq2)

**Objective**
Identify genes differentially expressed upon Influenza infection within
functionally defined T-cell states using a pseudobulk approach.

**Input**
- Annotated T-cell object from Notebook 08
- Sample-level metadata (`sample_id`)

**Methods**
- Aggregate raw counts per sample and T-cell subtype (pseudobulk)
- Differential expression analysis using DESeq2
- Separate comparisons:
  - Adults: IA vs HC
  - Pregnancy: PI vs PHC
- Analyses performed independently for selected T-cell states
  (e.g. Activated-like T cells, Cytotoxic T cells)

**Output**
- Differential expression tables for each comparison
- Summary figures (volcano / MA plots, selected gene signatures)

**Status**

Pseudobulk DESeq2 analysis was successfully completed for Activated-like T cells.

Outputs:
- `results/pseudobulk/deseq2_Activated_like_T_IA_vs_HC.csv`
- `results/pseudobulk/deseq2_Activated_like_T_PI_vs_PHC.csv`

Next step (reporting): summarize key differentially expressed genes and integrate results into the project report.

In [1]:
import scanpy as sc

# Load the analysis object you will use for pseudobulk (T cells or a T-cell subtype)
adata = sc.read_h5ad("../results/adata_tcells_annotated.h5ad")  # adjust if your file name differs

print("Cells:", adata.n_obs, "| Genes:", adata.n_vars)
print("\nColumns in adata.obs:")
print(list(adata.obs.columns))

# Try to detect likely sample/patient columns
candidates = [c for c in adata.obs.columns if any(k in c.lower() for k in ["patient", "donor", "sample", "orig", "batch"])]
print("\nCandidate sample/patient columns:", candidates)

# Also show counts for the most likely ones (if present)
for c in candidates[:5]:
    print("\n", c)
    print(adata.obs[c].value_counts().head(10))

Cells: 6044 | Genes: 1103

Columns in adata.obs:
['gsm', 'sample_id', 'condition', 'replicate', 'batch', 'leiden', 'cell_type', 'leiden_t', 'tcell_subtype']

Candidate sample/patient columns: ['sample_id', 'batch']

 sample_id
sample_id
PHC_2    1259
PHC_1    1144
HC_1     1133
HC_2      767
PI_2      554
IA_2      391
PI_1      358
IA_1      331
PI_3      107
Name: count, dtype: int64

 batch
batch
PHC_2    1259
PHC_1    1144
HC_1     1133
HC_2      767
PI_2      554
IA_2      391
PI_1      358
IA_1      331
PI_3      107
Name: count, dtype: int64


In [ ]:
# DEPRECATED:
# This pseudobulk was generated from normalized/log data.
# Kept for reference only. Do NOT use for DESeq2.
# Pseudobulk for Activated_like_T
import numpy as np
import pandas as pd
import scanpy as sc
from pathlib import Path
import scipy.sparse as sp

# --- Settings ---
SUBTYPE = "Activated_like_T"
SAMPLE_COL = "sample_id"
CONDITION_COL = "condition"

outdir = Path("../results/pseudobulk")
outdir.mkdir(parents=True, exist_ok=True)

# --- 1) Subset cells to the target T-cell state ---
adata_pb = adata[adata.obs["tcell_subtype"] == SUBTYPE].copy()
print(f"{SUBTYPE} cells:", adata_pb.n_obs, "| genes:", adata_pb.n_vars)

# Keep only conditions used in the project
adata_pb = adata_pb[adata_pb.obs[CONDITION_COL].isin(["HC", "IA", "PHC", "PI"])].copy()
print("After filtering conditions:", adata_pb.obs[CONDITION_COL].value_counts().to_dict())

# --- 2) Pick a counts matrix to aggregate (prefer raw counts) ---
# Priority: layers["counts"] > adata.raw.X > adata.X (last resort)
use_layer = None
if "counts" in adata_pb.layers:
    use_layer = "counts"
    X = adata_pb.layers["counts"]
    gene_names = adata_pb.var_names
    print("Using adata.layers['counts'] for pseudobulk.")
elif adata_pb.raw is not None:
    X = adata_pb.raw.X
    gene_names = adata_pb.raw.var_names
    print("Using adata.raw.X for pseudobulk.")
else:
    X = adata_pb.X
    gene_names = adata_pb.var_names
    print("WARNING: Using adata.X for pseudobulk (may be normalized/logged). Prefer raw counts if available.")

# --- 3) Aggregate counts per sample_id (sum across cells) ---
samples = adata_pb.obs[SAMPLE_COL].astype(str).values
unique_samples = pd.Index(sorted(pd.unique(samples)), name=SAMPLE_COL)

# Build a (n_samples x n_cells) one-hot mapping matrix to sum efficiently
sample_codes = pd.Categorical(samples, categories=unique_samples).codes  # 0..n_samples-1
n_samples = len(unique_samples)
n_cells = adata_pb.n_obs

# Create sparse mapping matrix M where M[sample, cell] = 1
M = sp.csr_matrix((np.ones(n_cells), (sample_codes, np.arange(n_cells))), shape=(n_samples, n_cells))

# Ensure X is sparse CSR for efficient multiplication
if not sp.issparse(X):
    X = sp.csr_matrix(X)
else:
    X = X.tocsr()

# Pseudobulk counts: (n_samples x n_genes)
PB = M @ X  # sums counts per sample
print("Pseudobulk matrix shape:", PB.shape)

# Convert to DataFrame (genes as columns, samples as rows)
counts_df = pd.DataFrame.sparse.from_spmatrix(PB, index=unique_samples, columns=gene_names)

# --- 4) Build sample-level metadata (one row per sample) ---
meta = (
    adata_pb.obs[[SAMPLE_COL, CONDITION_COL]]
    .drop_duplicates(subset=[SAMPLE_COL])
    .set_index(SAMPLE_COL)
    .loc[unique_samples]  # align order
)

# Basic sanity checks
print("\nSamples per condition:")
print(meta[CONDITION_COL].value_counts())

# --- 5) Save outputs ---
counts_path = outdir / f"pseudobulk_counts_{SUBTYPE}.tsv"
meta_path   = outdir / f"pseudobulk_meta_{SUBTYPE}.tsv"

counts_df.to_csv(counts_path, sep="\t")
meta.to_csv(meta_path, sep="\t")

print("\nSaved:")
print(" -", counts_path)
print(" -", meta_path)

Activated_like_T cells: 1033 | genes: 1103
After filtering conditions: {'PI': 519, 'IA': 448, 'PHC': 42, 'HC': 24}
Pseudobulk matrix shape: (9, 1103)

Samples per condition:
condition
PI     3
HC     2
IA     2
PHC    2
Name: count, dtype: int64

Saved:
 - ../results/pseudobulk/pseudobulk_counts_Activated_like_T.tsv
 - ../results/pseudobulk/pseudobulk_meta_Activated_like_T.tsv


In [2]:
# Pseudobulk reference code 
import scanpy as sc
import pandas as pd
import numpy as np
from pathlib import Path
import scipy.sparse as sp

# --- Paths (adjust only if your filenames differ) ---
raw_path  = "../results/adata_with_condition_raw.h5ad"   # raw counts object
tcell_path = "../results/adata_tcells_annotated.h5ad"    # has tcell_subtype + sample_id + condition

outdir = Path("../results/pseudobulk")
outdir.mkdir(parents=True, exist_ok=True)

# --- Load ---
adata_raw = sc.read_h5ad(raw_path)
adata_t   = sc.read_h5ad(tcell_path)

# --- Enforce unique cell indices (prevents InvalidIndexError) ---
adata_raw.obs_names_make_unique()
adata_t.obs_names_make_unique()

# --- Intersect cells safely ---
common = adata_raw.obs_names.intersection(adata_t.obs_names)
print("Common cells:", len(common))

# --- Subset raw to common cells ---
adata_raw_sub = adata_raw[common].copy()

# --- Copy needed annotations from T-cell object ---
cols = ["tcell_subtype", "sample_id", "condition"]
missing = [c for c in cols if c not in adata_t.obs.columns]
if missing:
    raise ValueError(f"Missing columns in T-cell object: {missing}")

# Use a DataFrame aligned to 'common' to avoid reindexing issues
ann = adata_t.obs.loc[common, cols].copy()
adata_raw_sub.obs[cols] = ann.values

# --- Subset to Activated-like T cells (RAW counts) ---
SUBTYPE = "Activated_like_T"
adata_act_raw = adata_raw_sub[adata_raw_sub.obs["tcell_subtype"] == SUBTYPE].copy()
print("Activated_like_T raw cells:", adata_act_raw.n_obs, "| genes:", adata_act_raw.n_vars)

# --- Build pseudobulk counts per sample_id (SUM raw counts) ---
X = adata_act_raw.X
if not sp.issparse(X):
    X = sp.csr_matrix(X)
else:
    X = X.tocsr()

samples = adata_act_raw.obs["sample_id"].astype(str).values
unique_samples = pd.Index(sorted(pd.unique(samples)), name="sample_id")

codes = pd.Categorical(samples, categories=unique_samples).codes
M = sp.csr_matrix(
    (np.ones(len(samples)), (codes, np.arange(len(samples)))),
    shape=(len(unique_samples), len(samples))
)

PB = M @ X  # (n_samples x n_genes)

counts_df = pd.DataFrame.sparse.from_spmatrix(
    PB, index=unique_samples, columns=adata_act_raw.var_names
)

meta_df = (
    adata_act_raw.obs[["sample_id", "condition"]]
    .drop_duplicates("sample_id")
    .set_index("sample_id")
    .loc[unique_samples]
)

# --- Save (overwrite) ---
counts_out = outdir / "pseudobulk_counts_Activated_like_T.tsv"
meta_out   = outdir / "pseudobulk_meta_Activated_like_T.tsv"

counts_df.to_csv(counts_out, sep="\t")
meta_df.to_csv(meta_out, sep="\t")

print("Saved:")
print(" -", counts_out)
print(" -", meta_out)

# --- Sanity check: total counts per sample must be > 0 ---
totals = np.array(PB.sum(axis=1)).ravel()
print("Total raw counts per sample (all should be > 0):")
print(pd.Series(totals, index=unique_samples))


/home/yasmina/miniconda3/lib/python3.13/site-packages/anndata/_core/anndata.py:1806: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


Common cells: 6044
Activated_like_T raw cells: 1033 | genes: 37493
Saved:
 - ../results/pseudobulk/pseudobulk_counts_Activated_like_T.tsv
 - ../results/pseudobulk/pseudobulk_meta_Activated_like_T.tsv
Total raw counts per sample (all should be > 0):
sample_id
HC_1      29775.0
HC_2      35661.0
IA_1     430453.0
IA_2     921429.0
PHC_1    105823.0
PHC_2     41607.0
PI_1     467603.0
PI_2     813412.0
PI_3     210175.0
dtype: float64


In [5]:
# Display top 10 genes 
import pandas as pd

adults = pd.read_csv(
    "../results/pseudobulk/pseudobulk_deseq2_Activated_like_T_IA_vs_HC.csv",
    index_col=0
)

preg = pd.read_csv(
    "../results/pseudobulk/pseudobulk_deseq2_Activated_like_T_PI_vs_PHC.csv",
    index_col=0
)

print("Top genes (IA vs HC):")
print(adults[["log2FoldChange","pvalue","padj"]].head(10))

print("\nTop genes (PI vs PHC):")
print(preg[["log2FoldChange","pvalue","padj"]].head(10))


Top genes (IA vs HC):
        log2FoldChange        pvalue          padj
ISG15         5.590364  5.357085e-18  1.004453e-14
XAF1          4.366742  4.274686e-12  4.007518e-09
MT2A          3.392468  4.915377e-11  3.072111e-08
LY6E          3.408992  1.095240e-10  5.133936e-08
IFITM3        3.113429  3.885011e-10  1.456879e-07
IFIT3         5.707511  1.510252e-09  4.719539e-07
SAT1          3.984185  1.715418e-08  4.594870e-06
MX1           4.966540  9.556467e-08  2.239797e-05
IFI6          7.955643  1.244042e-07  2.591755e-05
IFI44L        7.884107  1.676517e-07  3.143469e-05

Top genes (PI vs PHC):
        log2FoldChange        pvalue          padj
ISG15         5.051370  1.864068e-38  3.495127e-35
LY6E          3.734318  5.756657e-24  5.396866e-21
IFITM3        3.171465  9.217677e-22  5.761048e-19
IFI6          4.627758  3.481794e-20  1.632091e-17
SAT1          2.526810  2.279929e-18  8.549734e-16
XAF1          3.043182  1.992107e-17  6.225334e-15
IFIT3         5.366687  2.672587e-14